# Document Extraction with Azure AI Document Intelligence and Azure OpenAI GPT-4o (Text Only)

This sample demonstrates how to extract structured data from any document using Azure AI Document Intelligence and Azure OpenAI GPT models.

![Data Extraction](../../../../images/extraction-document-intelligence-openai.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes specific extraction instruction for the type of document, and the Markdown content of the document.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from a document using Azure OpenAI's GPT-4o model.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.
- Use the [logprobs](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#request-body:~:text=False-,logprobs,-integer) parameter in an OpenAI request to determine the confidence of the extracted structured output.

## Useful Tips

- Combine this technique with a [page classification](../../classification/README.md) approach to reduce the number of pages to extract from to only those that match your criteria for extraction.

## Setup

### Import modules

This sample takes advantage of the following .NET dependencies:

- **Azure.AI.DocumentIntelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **Azure.AI.OpenAI** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **Azure.Identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local components are also used:

- [**Invoice**](../../modules/samples/models/Invoice.csx) to provide the expected structured output JSON schema for invoice documents.
- [**OpenAIStructuredOutputsHelpers**](../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx) to generate structured outputs from the OpenAI API and provide parsing functions for the response output.
- [**AccuracyEvaluator**](../../modules/samples/evaluation/AccuracyEvaluator.csx) to evaluate the output of the classification process with expected results.
- [**DocumentIntelligenceConfidence**](../../modules/samples/confidence/DocumentIntelligenceConfidence.csx) to calculate the confidence of the classification process based on the analysis result from the Azure AI Document Intelligence API request.
- [**OpenAIConfidence**](../../modules/samples/confidence/OpenAIConfidence.csx) to calculate the confidence of the classification process based on the `logprobs` response from the OpenAI API request using the `Microsoft.ML.Tokenizers` and `Microsoft.ML.Tokenizers.Data.O200kBase` libraries (the latter is required for tokenizers used for the GPT-4o model).
- [**DocumentProcessingResult**](../../modules/samples/models/DocumentProcessingResult.csx) to store the results of the classification process as a file.
- [**AppSettings**](../../modules/samples/AppSettings.csx) to access environment variables from the `.env` file.

In [18]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.AI.DocumentIntelligence, 1.0.0"
#r "nuget: DotNetEnv, 3.1.1"
#r "nuget: Microsoft.ML.Tokenizers, 1.0.2"
#r "nuget: Microsoft.ML.Tokenizers.Data.O200kBase, 1.0.2"

#!import ../../modules/samples/AppSettings.csx
#!import ../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx
#!import ../../modules/samples/helpers/StopwatchContext.csx
#!import ../../modules/samples/models/Invoice.csx
#!import ../../modules/samples/models/DocumentProcessingResult.csx
#!import ../../modules/samples/evaluation/AccuracyEvaluator.csx
#!import ../../modules/samples/confidence/OpenAIConfidence.csx
//#!import ../../modules/samples/confidence/DocumentIntelligenceConfidence.csx

using System;
using System.IO;
using System.Text.Json;
using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;
using Azure.AI.DocumentIntelligence;
using OpenAI;
using OpenAI.Chat;
using DotNetEnv;

Installed Packages Azure.AI.DocumentIntelligence, 1.0.0 Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.13.2 DotNetEnv, 3.1.1 Microsoft.ML.Tokenizers, 1.0.2 Microsoft.ML.Tokenizers.Data.O200kBase, 1.0.2

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create a client instance using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [19]:
string workingDir = Path.GetFullPath("../../../../");
AppSettings settings = new AppSettings(new Dictionary<string, string>(Env.Load(Path.Combine(workingDir, ".env"))));
string samplePath = Path.Combine(workingDir, "samples/dotnet/extraction/text/");
string sampleName = "document-extraction-gpt";

DefaultAzureCredential credential = new DefaultAzureCredential(
    new DefaultAzureCredentialOptions { 
        ExcludeWorkloadIdentityCredential = true,
        ExcludeAzureDeveloperCliCredential = true,
        ExcludeEnvironmentCredential = true,
        ExcludeManagedIdentityCredential = true,
        ExcludeAzurePowerShellCredential = true,
        ExcludeSharedTokenCacheCredential = true,
        ExcludeInteractiveBrowserCredential = true
    }
);

AzureOpenAIClient openaiClient = new AzureOpenAIClient(
    new Uri(settings.OpenAIEndpoint),
    credential
);

var documentIntelligenceClient = new DocumentIntelligenceClient(
    new Uri(settings.AIServicesEndpoint),
    credential
);

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on the details of an [Invoice](../../../assets/invoices/invoice_1.pdf).

> **Note**: More invoice examples can be found in the [assets folder](../../../assets/invoices). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

The expected output has been defined by a human evaluating the document.

In [20]:
string path = Path.Combine(workingDir, "samples/assets/invoices/");
string metadataFName = "invoice_1.json"; // Change this to the file you want to evaluate
string metadataFPath = Path.Combine(path, metadataFName);

Dictionary<string, JsonElement> data = JsonSerializer.Deserialize<Dictionary<string, JsonElement>>(File.ReadAllText(metadataFPath));

Invoice expected = data["expected"].Deserialize<Invoice>(
    new JsonSerializerOptions(JsonSerializerDefaults.Web)
    {
        PropertyNamingPolicy = JsonNamingPolicy.SnakeCaseLower
    }
);
string pdfFName = data["fname"].GetString();
string pdfFPath = Path.Combine(path, pdfFName);

AccuracyEvaluator<Invoice> invoiceEvaluator = new AccuracyEvaluator<Invoice>(matchKeys: new List<string>() { "ProductCode", "Description" });

## Extract data from the document

The following code block executes the data extraction process using Azure AI Document Intelligence and Azure OpenAI's GPT-4o model.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the Markdown.

In [21]:
AnalyzeResult result;

StopwatchContext diSw;

using (diSw = new StopwatchContext())
{
    var pollerResult = await documentIntelligenceClient.AnalyzeDocumentAsync(
        WaitUntil.Completed,
        modelId: "prebuilt-layout",
        bytesSource: BinaryData.FromBytes(File.ReadAllBytes(pdfFPath))
    );

    result = pollerResult.Value;
}

string markdown = result.Content;

In [22]:
string systemPrompt = "You are an AI assistant that extracts data from documents.";

In [23]:
List<ChatMessageContentPart> userContent = new List<ChatMessageContentPart>();

In [24]:
StringBuilder userTextPromptBuilder = new StringBuilder();
userTextPromptBuilder.AppendLine("Extract the data from this invoice.");
userTextPromptBuilder.AppendLine("- If a value is not present, provide null.");
userTextPromptBuilder.AppendLine("- Dates should be in the format YYYY-MM-DD.");

string userTextPrompt = userTextPromptBuilder.ToString();

userContent.Add(ChatMessageContentPart.CreateTextPart(userTextPrompt));
userContent.Add(ChatMessageContentPart.CreateTextPart(markdown));

In [25]:
ParsedChatCompletion<Invoice> completion;

StopwatchContext oaiSw;

using (oaiSw = new StopwatchContext())
{
    completion = await openaiClient
        .GetChatClient(settings.GPT4OModelDeploymentName)
        .CompleteChatAsync(
            [
                new SystemChatMessage(systemPrompt),
                new UserChatMessage(userContent)
            ],
            new ChatCompletionOptions
            {   
                ResponseFormat = CreateJsonSchemaFormat<Invoice>("invoice", jsonSchemaIsStrict: true),
                MaxOutputTokenCount = 4096,
                Temperature = 0.1f,
                TopP = 0.1f,
                IncludeLogProbabilities = true
            }
        );
}

### Understanding the Structured Outputs JSON schema

Using [`OpenAIJsonSchema.For`](../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx), the [Invoice](../../modules/samples/models/Invoice.csx) data model is converted to a valid OpenAI supported JSON schema when applied to the `ResponseFormat` parameter of the OpenAI chat completions request.

The JSON schema is used to instruct the GPT-4o model to generate a strict output that adheres to the structure defined. The approach makes it easier for developers to manage the data structure in code, with helpful descriptions that will be included in the final JSON schema.

Demonstrated below, you can see how the Invoice data model is understood by the OpenAI request:

In [26]:
display(typeof(Invoice).ModelJsonSchema());

{
  "type": "object",
  "properties": {
    "CustomerName": {
      "type": "string",
      "description": "Name of the customer being invoiced, e.g. Company A"
    },
    "CustomerAddress": {
      "anyOf": [
        {
          "$ref": "#/$defs/InvoiceAddress"
        },
        {
          "type": "null"
        }
      ],
      "description": "Full address of the customer, e.g. 123 Main St., City, Country"
    },
    "CustomerTaxId": {
      "type": "string",
      "description": "Government tax ID of the customer, e.g. 123456789"
    },
    "ShippingAddress": {
      "anyOf": [
        {
          "$ref": "#/$defs/InvoiceAddress"
        },
        {
          "type": "null"
        }
      ],
      "description": "Full address of the shipping location for the customer (null if the same as customer address), e.g. 123 Main St., City, Country"
    },
    "PurchaseOrder": {
      "type": "string",
      "description": "Purchase order reference number, e.g. PO-1234"
    },
    "Invoic

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The Markdown representation of the document structure as determined by Azure AI Document Intelligence.
- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction by comparing against the Azure AI Document Intelligence analysis.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The side-by-side comparison of the expected output and the output generated by Azure OpenAI's GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [27]:
// Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(markdown);

NEXGEN
Innovation drives progress
INVOICE
Customer:
Sharp Consulting
Address:
73 Regal Way, Leeds, LS1 5AB, UK
Delv. Date:
5/16/2024
Invoice Number:
3847193
Purchase Order:
15931
Item Code
Item Desc.
Unit Price
Quantity
Total
MA197
STRETCHWRAP ROLL
16.62
5
83.10
ST4086
BALLPOINT PEN MED.
2.49
10
24.90
JF9912413BF
BUBBLE FILM ROLL CL.
15.46
12
185.52
NOTES
Total Pcs .:
27
Total Price:
293.52
Payable on or before 5/24/2024
Sfr
Cust. Sig.
Drivr. Sig.
Cust. Name:
Sarah H
Drivr. Name:
James T
NEXGEN Innovation drives progress
RETURNS
Customer:
Sharp Consulting
Address:
73 Regal Way, Leeds, LS1 5AB, UK
Coll. Date:
5/16/2024
Item Code
Item Desc.
Pcs.
Reason
This item was provided in previous order
MA145
POSTAL TUBE BROWN
1
as replacement
JF7902
MAILBOX 25PK
1 7
Not required
NOTES
Total Pcs .:
2
8
Cust. Sig.
Drivr. Sig.
&
Cust. Name:
Sarah H
Drivr. Name:
James T

In [28]:
// Gets the parsed Invoice object from the completion response.
var invoice = completion.Parsed;

In [29]:
// Determines the accuracy of the extracted data against the expected values.
var accuracy = invoiceEvaluator.Evaluate(expected, invoice);

In [30]:
// Determines the confidence of the extracted data using the OpenAI response.
var oaiConfidence = OpenAIConfidence<Invoice>.EvaluateConfidence(invoice, completion.Origin);

In [31]:
// Gets the total execution time of the data extraction process.
var totalElapsed = diSw.Elapsed + oaiSw.Elapsed;

// Gets the prompt tokens and completion tokens from the completion response.
var promptTokens = completion.Usage.InputTokenCount;
var completionTokens = completion.Usage.OutputTokenCount;

In [32]:
// Save the output of the data extraction result
var extractionResult = new DataProcessingResult<Invoice>(
    invoice,
    accuracy,
    oaiConfidence,
    promptTokens,
    completionTokens,
    totalElapsed
);

var extractionResultJson = JsonSerializer.Serialize(extractionResult, new JsonSerializerOptions { WriteIndented = true });
var extractionResultFPath = Path.Combine(samplePath, $"{sampleName}.{pdfFName}.json");

await File.WriteAllTextAsync(extractionResultFPath, extractionResultJson);

In [33]:
// Display the outputs of the extraction process.
var output = new
{
    Accuracy = $"{float.Parse(accuracy["overall"].ToString()) * 100:0.00}%",
    Confidence = $"{float.Parse(oaiConfidence["_overall"].ToString()) * 100:0.00}%",
    ExecutionTime = $"{totalElapsed.TotalSeconds:0.00} seconds",
    DocumentIntelligenceExecutionTime = $"{diSw.Elapsed.TotalSeconds:0.00} seconds",
    OpenAIExecutionTime = $"{oaiSw.Elapsed.TotalSeconds:0.00} seconds",
    PromptTokens = promptTokens,
    CompletionTokens = completionTokens,
};

display(output);
display(oaiConfidence);

Accuracy,70.27%
Confidence,57.72%
ExecutionTime,25.88 seconds
DocumentIntelligenceExecutionTime,6.42 seconds
OpenAIExecutionTime,19.45 seconds
PromptTokens,1660
CompletionTokens,444


key type value CustomerName System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999934290129487 value System.String Sharp Consulting CustomerAddress System.Collections.Generic.Dictionary<System.String,System.Object> key value Street key type value confidence System.Double 0.999969449653791 value System.String 73 Regal Way City key type value confidence System.Double 0.9999977203257707 value System.String Leeds State key type value confidence System.Double 0 value System.String PostalCode key type value confidence System.Double 0.9999240514903694 value System.String LS1 5AB Country key type value confidence System.Double 0.9987872872269569 value System.String UK CustomerTaxId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String ShippingAddress System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String PurchaseOrder System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9999896145282455 value System.String 15931 InvoiceId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.999955463616486 value System.String 3847193 InvoiceDate System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9998232841255587 value System.String 2024-05-16 PayableBy System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.999973124994913 value System.String 2024-05-24 VendorName System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9977037811794892 value System.String NEXGEN VendorAddress System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String VendorTaxId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String RemittanceAddress System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String Subtotal System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String TotalDiscount System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String TotalTax System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String InvoiceTotal System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0.9998907159408708 value System.String 293.52 PaymentTerms System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Double 0 value System.String Items System.Collections.Generic.List<System.Object> index value 0 key value ProductCode key type value confidence System.Double 0.9999870516794975 value System.String MA197 ProductDescription key type value confidence System.Double 0.9991328084874701 value System.String STRETCHWRAP ROLL Quantity key type value confidence System.Double 0.9998926480548848 value System.String 5 Tax key type value confidence System.Double 0 value System.String TaxRate key type value confidence System.Double 0 value System.String UnitPrice key type value confidence System.Double 0.9999823233393479 value System.String 16.62 Total key type value confidence System.Double 0.8903329046351649 value System.String 83.1 Reason key type value confidence System.Double 0.7057758780731179 value System.String 1 key value ProductCode key type value confidence System.Double 0.9999944025036939 value System.String ST4086 ProductDescription key type value confidence System.Double 0.9994897414681014 value System.String BALL